In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model, Sequential

import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

import random
random.seed(1228)

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

%matplotlib inline

Using TensorFlow backend.


я бы поставила семь, конечно, но можно и три пока а это для последнего задания

In [2]:
TEXT_LENGTH = 1800
VOCABULARY_SIZE = 250000
EMBEDDING_DIM = 300
DIMS = 250
MAX_FEATURES = 5000
batch_size = 32

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 10

In [3]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_corpus.body)
tokenizer

NameError: name 'train_corpus' is not defined

### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

Reading the data:

In [4]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [9]:
columns = ['ignore', 'words', 'ignore', 'pos', 'chunk']
train_reader = ConllCorpusReader(root = '.', fileids = 'unamb_sent_14_6.conllu', columntypes = columns)


sents = list(train_reader.iob_sents())
train_sents = sents[:30000]
test_sents = sents[30000:]

print(len(train_sents))
print(len(test_sents))

30000
8508


In [36]:
#вытащим все тэги и посчитаем их кол-во
corpus = []
d = {}
for i in sents[:10]:
    sentence = []
    for j in range(0, len(i)):
        if i[j][1] not in d:
            d[i[j][1]] = 1
        else:
            d[i[j][1]] += 1
        sentence.append(i[j][1])
    corpus.append(sentence)

In [40]:
print(corpus[:2])
# ну вроде всё получилось

[['PUNCT', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'VERB', 'NOUN'], ['VERB', 'PART', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT']]


In [44]:
# сформируем тестовую и тренировочную выборки для тэгов
train_tags = sents[:30000]
test_tags = sents[30000:]

print(len(train_tags))
print(len(test_tags))

30000
8508


In [39]:
print(d) # а теперь мы увидели, сколько каких частей речи встречается в корпусе. хе-хе-хе

{'PART': 7, 'CONJ': 9, 'ADV': 5, 'ADJ': 18, 'PUNCT': 42, 'DET': 6, 'X': 1, 'NOUN': 48, 'PROPN': 9, 'PRON': 3, 'VERB': 13, 'ADP': 23}


In [41]:
len(train_reader.sents()) # Предложений в корпусе 38508

38508

In [42]:
sequences = tokenizer.texts_to_sequences(train_s.body)
X_train = tokenizer.sequences_to_matrix(sequences, mode='count')
sequences = tokenizer.texts_to_sequences(test_corpus.body)
X_test = tokenizer.sequences_to_matrix(sequences, mode='count')

NameError: name 'train_s' is not defined

Посмотреть какая именно архитектура сенны

In [ ]:
model = Sequential()
model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=TEXT_LENGTH))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))
model.add(Dropout(0.5))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)